Blah

In [9]:
# Define the Renderer and Processing Thread
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
from threading import Thread
import numpy as np
import time
from IPython.display import display, Image
from moviepy.editor import ImageSequenceClip
from collections import namedtuple
import math
from collections.abc import Callable

out = Output()

# 1000/125 = 8 FPS = 0.125
# 1000/40 = 25 FPS = 0.04

TARGET_FPS = 25
SLEEP_TIME_SEC:float = 0.04

Ball = namedtuple('Ball', ['x', 'y', 'radius', 'fill_style', 'stroke','rotation_distance', 'speed'])

class Renderer:
  def __init__(self, canvas, save_image, ):
    self.frames = []
    self.canvas = canvas
    self.canvas.observe(self.get_array, 'image_data')
    self.setup_canvas()
    self.save_image = save_image
    self.ball = Ball(canvas.width/2, canvas.height/2, 10, 'blue', 'black', 50, 0.5)
    self.ball_travel = 0
    self._render_steps = []

  def render_steps(self, render_steps: list[Callable[...,None]]) -> None:
    self._render_steps = render_steps
    
  @out.capture()
  def get_array(self, *args, **kwargs):
    if self.save_image:
      arr = self.canvas.get_image_data()
      self.frames.append(arr)

  def setup_canvas(self):
    pass
    
  def render(self, tick):
    with hold_canvas(self.canvas):
      for step in self._render_steps:
        step(canvas, self)
      
class RenderingThread(Thread):
  def __init__(self, total_frames, renderer, image_name):
    super(RenderingThread, self).__init__()
    self.total_frames = total_frames
    self.renderer = renderer
    self.image_name = image_name

  def run(self):
    for i in range(self.total_frames):
      self.renderer.render(i)
      time.sleep(SLEEP_TIME_SEC)
    self.on_complete()    

  def on_complete(self):
    if self.renderer.save_image:
      self.save_gif()
        
  @out.capture()
  def save_gif(self):
    print(f"Number of frames: {len(self.renderer.frames)}")
    clip = ImageSequenceClip(list(self.renderer.frames), fps=TARGET_FPS)
    clip.write_gif(self.image_name, fps=TARGET_FPS)
    # Image(self.image_name)

In [10]:
# The Main Entry Point

# Set this to False to do development in VSCode. 
# Set to True when running in Jupyter Notebooks
SAVE_ANIMATED_GIF = False

def clear(canvas, context):
  canvas.fill_style = 'white'
  canvas.fill_rect(0, 0, canvas.width, canvas.height)
  
def draw_ball(canvas, context):
  canvas.fill_style = context.ball.fill_style
  context.ball_travel += context.ball.speed
  x_offset = math.cos(context.ball_travel) * context.ball.rotation_distance
  y_offset = -1 * math.sin(context.ball_travel) * context.ball.rotation_distance
  canvas.fill_circle(int(x_offset) + context.ball.x, int(y_offset) + context.ball.y, context.ball.radius)

display(out)
canvas = Canvas(width=200, height=200, sync_image_data=True)
renderer = Renderer(canvas, SAVE_ANIMATED_GIF)
renderer.render_steps([clear, draw_ball])
thread = RenderingThread(100, renderer, 'test.gif')

display(canvas)
thread.start()

Output()

Canvas(height=200, sync_image_data=True, width=200)